## 1: Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv('/kaggle/input/bank-marketing/bank-additional-full.csv', sep=';')
df

##  2: Exploratory Data Analysis (EDA)

In [ ]:
df.info()

In [ ]:
# Check for missing values
df.isnull().sum()

In [ ]:
# Separate numerical and categorical columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object', 'bool']).columns

# Plot histograms for categorical columns
for col in categorical_cols:
    plt.figure(figsize=(6, 3))
    df[col].value_counts().plot(kind='bar')
    plt.title(col)
    plt.show()

# Plot histograms + box plots for numerical columns
for col in numerical_cols:
    plt.figure(figsize=(6, 3))
    
    # Histogram
    plt.subplot(1, 2, 1)
    df[col].hist(bins=30)
    plt.title(col)
    
    # Box plot
    plt.subplot(1, 2, 2)
    df[col].plot(kind='box')
    
    plt.show()

In [ ]:
#Remove Unwanted Features
df.drop(columns=['day_of_week'], inplace=True)
df.info()

## 3: Data Preprocessing

In [ ]:
# Apply one-hot encoding to 'job', 'marital', and 'contact'
df = pd.get_dummies(df, columns=['job', 'marital', 'contact'], drop_first=True)

# Define mapping for ordinal encoding
ordinal_mapping = {
    'education': {'basic.4y': -1, 'basic.6y': -1, 'basic.9y': -1, 'high.school': 1,
                  'illiterate': -2, 'professional.course': 1, 'university.degree': 2, 'unknown': 0},
    'month': {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
              'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12},
    'poutcome': {'failure': -1, 'nonexistent': 0, 'success': 1},
    'default': {'no': -1, 'yes': 1, 'unknown': 0},
    'housing': {'no': -1, 'yes': 1, 'unknown': 0},
    'loan': {'no': -1, 'yes': 1, 'unknown': 0}
}

# Apply ordinal encoding
for column, mapping in ordinal_mapping.items():
    df[column] = df[column].map(mapping)

# Apply binary encoding for the 'y' column (output variable)
df['y'] = df['y'].map({'no': 0, 'yes': 1})

df

In [ ]:
df.describe()

In [ ]:
# Calculate correlations
corr = df.corr()

# Use seaborn to plot the heatmap
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm')

plt.show()

In [ ]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform all numerical columns
df[df.columns] = scaler.fit_transform(df[df.columns])

# Display the preprocessed DataFrame
print(df.head())

## 4: Split Data into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('y', axis=1)  # Features
y = df['y']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 5 : Build and Train the Neural Network Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


## 6 : Evaluate the Model

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


In [ ]:
# Generate predictions using the trained model
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)  

# Create a confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted 0', 'Predicted 1'], 
            yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()